<div style="text-align: center;">
    <h1> <font style="bold"> Trabajo Práctico Integrador </font></h1>
    <h2><font style="bold">Visión por Computadora II - CEIA </font></h2>
    <h3><font style="bold">Abril Noguera - Pedro Barrera - Ezequiel Caamaño</font></h3>
    <h4><font style="bold">Modelo Baseline</font></h4>
</div>

# Detección Automatizada de Cáncer de Pulmón mediante Visión por Computadora

## Modelo Baseline

### Diseño

Para construir un modelo base con buen rendimiento inicial y poco esfuerzo de desarrollo, se optó por usar una arquitectura preentrenada con **transfer learning**.

- **Modelo preentrenado**: `ResNet18`, ya entrenado en ImageNet
  - Arquitectura liviana y eficiente
  - Buen balance entre precisión y velocidad
- **Técnica**: Transfer learning con mínimo fine-tuning
  - Se congelan los pesos de todas las capas convolucionales
  - Solo se entrena el **head** (la capa final de clasificación)
- **Salida del modelo**: Clasificación multiclase con 4 clases:
  - `adenocarcinoma`, `large.cell.carcinoma`, `squamous.cell.carcinoma`, `normal`

Esta configuración busca obtener una primera línea de base rápida, sin necesidad de entrenar el modelo desde cero.

### Importación de librerias

In [10]:
import os
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np
from torchvision.models import resnet18, ResNet18_Weights

from dotenv import load_dotenv

# Cargar las variables desde el archivo .env
load_dotenv()

True

### Configuración

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path = os.getenv("DATASET_PATH")
data_dir = os.path.join(path, "Data")
num_classes = 4
batch_size = 32
num_epochs = 5
learning_rate = 1e-3

In [11]:
# Transformaciones para los datos
weights = ResNet18_Weights.DEFAULT
transform = weights.transforms()

print(transform)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [12]:
# Datasets y DataLoaders
train_ds = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
valid_ds = datasets.ImageFolder(os.path.join(data_dir, "valid"), transform=transform)
test_ds  = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_ds, batch_size=batch_size)
test_loader  = DataLoader(test_ds, batch_size=batch_size)

In [15]:
# Modelo
model = models.resnet18(pretrained=True)

# Congelar capas convolucionales
for param in model.parameters():
    param.requires_grad = False

# Reemplazar la capa final
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

print(model)

/Users/anoguera/anaconda3/envs/CEIA/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/anoguera/anaconda3/envs/CEIA/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Entrenamiento

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.fc.parameters(), lr=learning_rate)

def evaluar(modelo, dataloader):
    modelo.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            outputs = modelo(x)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    return acc

print("Entrenamiento:")

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluación por epoch
    train_acc = evaluar(model, train_loader)
    val_acc = evaluar(model, valid_loader)
    print(f"Epoch {epoch+1}/{num_epochs} | Train Acc: {train_acc:.2f} | Val Acc: {val_acc:.2f}")


Entrenamiento:
Epoch 1/5 | Train Acc: 0.55 | Val Acc: 0.54
Epoch 2/5 | Train Acc: 0.61 | Val Acc: 0.57
Epoch 3/5 | Train Acc: 0.66 | Val Acc: 0.54
Epoch 4/5 | Train Acc: 0.72 | Val Acc: 0.57
Epoch 5/5 | Train Acc: 0.75 | Val Acc: 0.56


In [18]:
test_acc = evaluar(model, test_loader)
print(f"Precisión en Test: {test_acc:.2f}")

Precisión en Test: 0.61


In [ ]:
# Guardar el modelo entrenado
torch.save(model.state_dict(), "baseline_model.pth")

### Conclusiones Preliminares